In [1]:
from src.models.Encodec import Encodec
encodec = Encodec()


/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
from audiocraft.modules.codebooks_patterns import DelayedPatternProvider

pattern_provider = DelayedPatternProvider(4)

import torch
test = torch.randn(1,24000*20)
codes = encodec(test)


In [3]:
codes.shape

torch.Size([1, 4, 750])

In [3]:
pattern = pattern_provider.get_pattern(codes.shape[-1])

In [4]:
from src.models.model import MuMRVQ
from src.models.Transformer.Encoder import VanillaEncoder
model = MuMRVQ(encodec, VanillaEncoder(), debug = True)

/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
model(test)

codes shape : torch.Size([1, 4, 225])
codes after pattern : torch.Size([1, 4, 229])


IndexError: index out of range in self